<a href="https://colab.research.google.com/github/taegeonyu/HDS-5230-07/blob/main/Week10/Week10_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    confusion_matrix
)

from sklearn.model_selection import train_test_split
import time
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

## Data Pre-processing

In [2]:
# load data
data = \
      pd.read_csv('/content/PatientAnalyticFile.csv')

In [3]:
# copy the original dataframe
df = data.copy()

In [4]:
# check 5 random samples
df.sample(n = 5, random_state = 1)

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
11456,11457,1959-07-06,male,hispanic,0,0,0,0,0,0,...,0,0,0,1,0,0,0,2016-05-07,2018-06-01,NaN
16528,16529,1993-09-04,female,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2010-03-06,2015-03-12,NaN
3253,3254,1977-04-02,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-05-12,2018-06-01,NaN
18614,18615,1966-11-26,female,white,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2015-04-21,2016-11-17,2016-11-17
1544,1545,1974-11-05,female,hispanic,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2009-02-06,2011-01-06,2011-01-06


In [5]:
# check dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   PatientID                       20000 non-null  int64 
 1   DateOfBirth                     20000 non-null  object
 2   Gender                          20000 non-null  object
 3   Race                            20000 non-null  object
 4   Myocardial_infarction           20000 non-null  int64 
 5   Congestive_heart_failure        20000 non-null  int64 
 6   Peripheral_vascular_disease     20000 non-null  int64 
 7   Stroke                          20000 non-null  int64 
 8   Dementia                        20000 non-null  int64 
 9   Pulmonary                       20000 non-null  int64 
 10  Rheumatic                       20000 non-null  int64 
 11  Peptic_ulcer_disease            20000 non-null  int64 
 12  LiverMild                       20000 non-null

In [6]:
# check duplicated rows
df.duplicated().sum()

np.int64(0)

* no duplicated rows confirmed

In [7]:
# check null values
df.isnull().sum()

,0
PatientID,0
DateOfBirth,0
Gender,0
Race,0
Myocardial_infarction,0
Congestive_heart_failure,0
Peripheral_vascular_disease,0
Stroke,0
Dementia,0
Pulmonary,0


* 12,906 null values confirmed in DateOfDeath, indicating alive patients.

In [8]:
# create new column 'mortality' as a target varaible (null  = 0 (alive) / not null = 1 (dead))
df['mortality'] = \
  np.where(df['DateOfDeath'].isnull(), 0, 1)
df['mortality'].value_counts(1)

,proportion
mortality,
0,0.6453
1,0.3547


In [9]:
# create new column age by calculating the time difference between reference date and date of birth and divide by 365.25
df['DateOfBirth'] = \
 pd.to_datetime(df['DateOfBirth'])

# patient's age including age of death
df['Age'] = \
  ((pd.to_datetime('2015-01-01') - df['DateOfBirth']).dt.days/365.25
  )

In [10]:
# drop unnecessary columns
df.drop(columns = ['PatientID','First_Appointment_Date','DateOfBirth',
                   'Last_Appointment_Date','DateOfDeath'], inplace = True)
df.columns

Index(['Gender', 'Race', 'Myocardial_infarction', 'Congestive_heart_failure',
       'Peripheral_vascular_disease', 'Stroke', 'Dementia', 'Pulmonary',
       'Rheumatic', 'Peptic_ulcer_disease', 'LiverMild',
       'Diabetes_without_complications', 'Diabetes_with_complications',
       'Paralysis', 'Renal', 'Cancer', 'LiverSevere',
       'Metastatic_solid_tumour', 'HIV', 'Obesity', 'Depression',
       'Hypertension', 'Drugs', 'Alcohol', 'mortality', 'Age'],
      dtype='object')

In [11]:
formula = "mortality ~ " + " + ".join([col for col in df.columns if col != 'mortality'])
formula

'mortality ~ Gender + Race + Myocardial_infarction + Congestive_heart_failure + Peripheral_vascular_disease + Stroke + Dementia + Pulmonary + Rheumatic + Peptic_ulcer_disease + LiverMild + Diabetes_without_complications + Diabetes_with_complications + Paralysis + Renal + Cancer + LiverSevere + Metastatic_solid_tumour + HIV + Obesity + Depression + Hypertension + Drugs + Alcohol + Age'

In [12]:
# check the values distributed in variables with object dtype
obj_cols = df.select_dtypes(include = 'object').columns.tolist()

for col in df[obj_cols]:
  print(df[col].unique())
  print('-' * 30)

['female' 'male']
------------------------------
['hispanic' 'white' 'black' 'other']
------------------------------


In [13]:
# create subsample of data for faster processing
df_sub = \
  df.sample(frac = 0.1, random_state = 1)

# use Patsy to create model matrices
y,X = dmatrices(formula,
                df_sub)

In [14]:
X

DesignMatrix with shape (2000, 28)
  Columns:
    ['Intercept',
     'Gender[T.male]',
     'Race[T.hispanic]',
     'Race[T.other]',
     'Race[T.white]',
     'Myocardial_infarction',
     'Congestive_heart_failure',
     'Peripheral_vascular_disease',
     'Stroke',
     'Dementia',
     'Pulmonary',
     'Rheumatic',
     'Peptic_ulcer_disease',
     'LiverMild',
     'Diabetes_without_complications',
     'Diabetes_with_complications',
     'Paralysis',
     'Renal',
     'Cancer',
     'LiverSevere',
     'Metastatic_solid_tumour',
     'HIV',
     'Obesity',
     'Depression',
     'Hypertension',
     'Drugs',
     'Alcohol',
     'Age']
  Terms:
    'Intercept' (column 0)
    'Gender' (column 1)
    'Race' (columns 2:5)
    'Myocardial_infarction' (column 5)
    'Congestive_heart_failure' (column 6)
    'Peripheral_vascular_disease' (column 7)
    'Stroke' (column 8)
    'Dementia' (column 9)
    'Pulmonary' (column 10)
    'Rheumatic' (column 11)
    'Peptic_ulcer_disease' (c

In [15]:
y_series = pd.Series(np.ravel(y))
y_series.value_counts(1)

,proportion
0.0,0.649
1.0,0.351


* The distribution of classifiers are approximately 65:35 percent

In [16]:
# split the dataset into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y), random_state = 1, test_size = 0.25)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1500, 28) (1500,) (500, 28) (500,)


* The data is now processed for models to use.

## 1. Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.

In [17]:
# function to check model performance
def model_performance_checker(model, predictors, target):
    # model prediction
    pred = model.predict(predictors)

    # matrix scores
    acc = accuracy_score(target, pred)
    precision = precision_score(target, pred)
    recall = recall_score(target, pred)
    f1 = f1_score(target, pred)

    # output
    perf_df = pd.DataFrame({
        'Accuracy': acc,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
    }, index = [0])

    return perf_df

In [18]:
# empty dictionaries to store the models
train_mod = {}
test_mod = {}

### Logistic Regression

In [19]:
# define and fit the model
lr_mod = LogisticRegression(solver = 'liblinear', random_state = 1).fit(X_train, y_train)
lr_mod

LogisticRegression(random_state=1, solver='liblinear')

In [20]:
# comparison of training and test data
lr_mod_train = model_performance_checker(lr_mod, X_train, y_train)
lr_mod_test = model_performance_checker(lr_mod, X_test, y_test)

print(lr_mod_train)
print('-' * 40)
print(lr_mod_test)

   Accuracy  Precision    Recall        F1
0  0.737333   0.652068  0.516378  0.576344
----------------------------------------
   Accuracy  Precision    Recall      F1
0     0.736   0.686131  0.513661  0.5875


* The model is not overfitting and showing consistency for all scores.
* However, the model is not performing well.

In [21]:
# add the performance metrics
train_mod['Logistic'] = lr_mod_train
test_mod['Logistic'] = lr_mod_test

## Dummy Classifier

In [22]:
# check the dummy variable
dum = DummyClassifier(strategy = 'most_frequent',random_state = 1).fit(X_train, y_train)
dum.score(X_train, y_train)

0.654

In [23]:
# comparison of training and test data
dum_train = model_performance_checker(dum, X_train, y_train)
dum_test = model_performance_checker(dum, X_test, y_test)

print(dum_train)
print('-' * 40)
print(dum_test)

   Accuracy  Precision  Recall   F1
0     0.654        0.0     0.0  0.0
----------------------------------------
   Accuracy  Precision  Recall   F1
0     0.634        0.0     0.0  0.0


* The lr model is performing better than dummy model in accuracy, not not considerably better.

In [24]:
# add the performance metrics
train_mod['Dummy'] = dum_train
test_mod['Dummy'] = dum_test

## Regularized Linear Regression

In [25]:
# define and fit the model: L1 penalty & C = 1
lr_mod_l1 = LogisticRegression(C = 1, penalty = 'l1', solver = 'liblinear', random_state = 1).fit(X_train, y_train)
lr_mod_l1

LogisticRegression(C=1, penalty='l1', random_state=1, solver='liblinear')

In [26]:
# comparison of training and test data
lr_mod_l1_train = model_performance_checker(lr_mod_l1, X_train, y_train)
lr_mod_l1_test = model_performance_checker(lr_mod_l1, X_test, y_test)

print(lr_mod_l1_train)
print('-' * 40)
print(lr_mod_l1_test)

   Accuracy  Precision    Recall        F1
0     0.736   0.648193  0.518304  0.576017
----------------------------------------
   Accuracy  Precision    Recall        F1
0     0.738   0.683099  0.530055  0.596923


* Overall model performance is slightly better than the original lr_mod, but does make big difference.
* Changing the C into 0.1 or 10 could make a difference.

In [27]:
# add the performance metrics
train_mod['Logistic_l1'] = lr_mod_train
test_mod['Logistic_l1'] = lr_mod_test

In [28]:
# define and fit the model: L1 penalty & C = 0.1
lr_mod_l1_C01 = LogisticRegression(C = 0.1, penalty = 'l1', solver = 'liblinear', random_state = 1).fit(X_train, y_train)
lr_mod_l1_C01

LogisticRegression(C=0.1, penalty='l1', random_state=1, solver='liblinear')

In [29]:
# comparison of training and test data
lr_mod_l1_C01_train = model_performance_checker(lr_mod_l1_C01, X_train, y_train)
lr_mod_l1_C01_test = model_performance_checker(lr_mod_l1_C01, X_test, y_test)

print(lr_mod_l1_C01_train)
print('-' * 40)
print(lr_mod_l1_C01_test)

   Accuracy  Precision    Recall        F1
0  0.733333   0.650633  0.495183  0.562363
----------------------------------------
   Accuracy  Precision    Recall      F1
0     0.736   0.686131  0.513661  0.5875


In [30]:
# add the performance metrics
train_mod['Logistic_l1_CO1'] = lr_mod_l1_C01_train
test_mod['Logistic_l1_C01'] = lr_mod_l1_C01_test

In [31]:
# define and fit the model: L1 penalty & C = 10
lr_mod_l1_C10 = LogisticRegression(C = 10, penalty = 'l1', solver = 'liblinear', random_state = 1).fit(X_train, y_train)
lr_mod_l1_C10

LogisticRegression(C=10, penalty='l1', random_state=1, solver='liblinear')

In [32]:
# comparison of training and test data
lr_mod_l1_C10_train = model_performance_checker(lr_mod_l1_C10, X_train, y_train)
lr_mod_l1_C10_test = model_performance_checker(lr_mod_l1_C10, X_test, y_test)

print(lr_mod_l1_C10_train)
print('-' * 40)
print(lr_mod_l1_C10_test)

   Accuracy  Precision    Recall        F1
0  0.735333   0.647343  0.516378  0.574491
----------------------------------------
   Accuracy  Precision    Recall        F1
0     0.732   0.676259  0.513661  0.583851


In [33]:
# add the performance metrics
train_mod['Logistic_l1_C10'] = lr_mod_l1_C10_train
test_mod['Logistic_l1_C10'] = lr_mod_l1_C10_test

In [34]:
# define and fit the model: L1 penalty & 5 cv
lr_mod_cv = LogisticRegressionCV(cv = 5, Cs = 20, penalty = 'l1', solver='liblinear').fit(X_train, y_train)
lr_mod_cv

LogisticRegressionCV(Cs=20, cv=5, penalty='l1', solver='liblinear')

In [35]:
# comparison of training and test data
lr_mod_cv_train = model_performance_checker(lr_mod_cv, X_train, y_train)
lr_mod_cv_test = model_performance_checker(lr_mod_cv, X_test, y_test)

print(lr_mod_cv_train)
print('-' * 40)
print(lr_mod_cv_test)

   Accuracy  Precision    Recall        F1
0      0.74    0.66005  0.512524  0.577007
----------------------------------------
   Accuracy  Precision    Recall        F1
0     0.754   0.714286  0.546448  0.619195


In [36]:
# add the performance metrics
train_mod['Logistic_CV'] = lr_mod_cv_train
test_mod['Logistic_CV'] = lr_mod_cv_test

## Random Forest

In [37]:
# define and fit the model: random forest
rf = RandomForestClassifier(n_estimators = 100, max_features = 10, random_state = 1).fit(X_train, y_train)
rf

RandomForestClassifier(max_features=10, random_state=1)

In [38]:
# comparison of training and test data
rf_train = model_performance_checker(rf, X_train, y_train)
rf_test = model_performance_checker(rf, X_test, y_test)

print(rf_train)
print('-' * 40)
print(rf_test)

   Accuracy  Precision    Recall        F1
0  0.999333        1.0  0.998073  0.999036
----------------------------------------
   Accuracy  Precision    Recall        F1
0       0.7   0.607843  0.508197  0.553571


In [39]:
# add the performance metrics
train_mod['RandomForest'] = rf_train
test_mod['RandomForest'] = rf_test

In [40]:
# tabulate the results of the models
for k, v in test_mod.items():
    print(f"\nModel: {k}")
    print("\n")
    print(tabulate(v, headers = "keys", tablefmt = "pretty", showindex = False))


Model: Logistic


+----------+--------------------+--------------------+--------+
| Accuracy |     Precision      |       Recall       |   F1   |
+----------+--------------------+--------------------+--------+
|  0.736   | 0.6861313868613139 | 0.5136612021857924 | 0.5875 |
+----------+--------------------+--------------------+--------+

Model: Dummy


+----------+-----------+--------+-----+
| Accuracy | Precision | Recall | F1  |
+----------+-----------+--------+-----+
|  0.634   |    0.0    |  0.0   | 0.0 |
+----------+-----------+--------+-----+

Model: Logistic_l1


+----------+--------------------+--------------------+--------+
| Accuracy |     Precision      |       Recall       |   F1   |
+----------+--------------------+--------------------+--------+
|  0.736   | 0.6861313868613139 | 0.5136612021857924 | 0.5875 |
+----------+--------------------+--------------------+--------+

Model: Logistic_l1_C01


+----------+--------------------+--------------------+--------+
| Accuracy | 

* Ovrall, it seems like Logistic Regression model with cross validation has the highest performance, specifically in Accuracy and Precision.

## 2. Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

In [41]:
# create new set of predictors and target (full data)
y_full, X_full = dmatrices(formula,
                df)
print(X_full.shape, y_full.shape)

(20000, 28) (20000, 1)


In [42]:
# ensuring the subsets are the same with 80 percent of the observation in training data and the rest to test data
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, random_state = 1, test_size = 0.2, stratify = y_full)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16000, 28) (4000, 28) (16000, 1) (4000, 1)


* The 80:20 ratio between training and test sets are correct.

In [43]:
# create dict to store all these results:
result_scores = {}

In [44]:
# list of solvers
solvers = ['liblinear', 'saga', 'sag', 'newton-cg', 'lbfgs']

# calculate the accuracy score for all solvers
for solver in solvers:
        # start time
        start_time = time.time()

        # define and fit the model

        # liblinear does not accept penalty as None, uses l2 regularization as default
        if solver == 'liblinear':
          model = LogisticRegression(solver = solver, random_state = 1)
          model.fit(X_train, y_train)
        # the rest of the solvers accept None
        else:
          model = LogisticRegression(solver = solver, penalty = None, random_state = 1)
          model.fit(X_train, y_train)

        # calculate time taken to train the model
        time_taken = time.time() - start_time

        # predictions made on training and test dat
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)

        # save the results - name, training perf, test perf, and time into result scores
        result_scores[f'Logistic-Regression ({solver})'] = (
            accuracy_score(y_train, train_pred),
            accuracy_score(y_test, test_pred),
            time_taken
        )

In [45]:
result_scores

{'Logistic-Regression (liblinear)': (0.7484375, 0.739, 0.14427828788757324),
 'Logistic-Regression (saga)': (0.74875, 0.739, 3.7063558101654053),
 'Logistic-Regression (sag)': (0.7485625, 0.7395, 3.406754732131958),
 'Logistic-Regression (newton-cg)': (0.7485625, 0.73875, 0.14921879768371582),
 'Logistic-Regression (lbfgs)': (0.7486875, 0.739, 0.2958040237426758)}

* The results are saved in result_scores dictionary.

## 3. Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function). Summarize your results via a table with the following structure:

| Solver used| Training subset accuracy | Holdout subset accuracy | Time taken |
|--------------------------------------|---------------------------|--------------------------|-------------|

In [46]:
# create dataframe with the model performance
model_comparison_df = pd.DataFrame.from_dict(
    result_scores,
    orient = 'index',
    columns = ['Training subset accuracy', 'Holdout subset accuracy', 'Time taken']
).sort_values(by = 'Time taken').reset_index()

# specify the columns on data frame
model_comparison_df.columns = ['Solver used', 'Training subset accuracy', 'Holdout subset accuracy', 'Time taken']

In [47]:
print('Model Performance Comparison:')
model_comparison_df

Model Performance Comparison:


,Solver used,Training subset accuracy,Holdout subset accuracy,Time taken
0,Logistic-Regression (liblinear),0.748437,0.73900,0.144278
1,Logistic-Regression (newton-cg),0.748563,0.73875,0.149219
2,Logistic-Regression (lbfgs),0.748687,0.73900,0.295804
3,Logistic-Regression (sag),0.748563,0.73950,3.406755
4,Logistic-Regression (saga),0.748750,0.73900,3.706356


## 4. Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?

Overall, the best solver for the Logistic Regression model with our training and test data is liblinear. While the performances of training and test accuracy is pretty much the same, the model with liblinear solver was the fastest among the models. However, unlike other models specificed not to convey penalties, liblinear solver used l2 regularization as default (does not accept None). In this case, among the solvers without regularization, newton-cg is the fastest.